# 0. 라이브러리 & 데이터

## 0.1 라이브러리

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

import warnings
warnings.simplefilter("ignore")

## 0.2 데이터

In [3]:
case_fraud = pd.read_csv('../1_Data/case_fraud_prep.csv')
case_fraud.head(1)

,index,판례일련번호,사건명,사건번호,선고일자,법원명,사건종류명,사건종류코드,판결유형,선고,판례상세링크,판시사항,판결요지,참조조문,참조판례,판례내용,판례내용_전처리,판례내용_전처리_한자,판례내용_전처리_글자수,판례내용_토큰화_코모란,판례내용_토큰화_꼬꼬마
0,20,238021,특정경제범죄가중처벌등에관한법률위반(사기)·사기,2023도12424,2023.11.16,대법원,형사,400102.0,판결,선고,/DRF/lawService.do?OC=yeorii&target=prec&ID=23...,"유사수신행위를 금지·처벌하는 유사수신행위의 규제에 관한 법률 제6조 제1항, 제3...","유사수신행위의 규제에 관한 법률(이하 ‘유사수신행위법’이라 한다) 제6조 제1항, ...","유사수신행위의 규제에 관한 법률 제3조, 제6조 제1항, 형법 제37조, 제347조...",내용없음,【피 고 인】 피고인【상 고 인】 피고인【변 호 인】 변호사 장용배【배상신청인】 배...,상고이유를 판단한다 불가벌적 사후행위에 대한 판단유사수신행위의 규제에 관한 ...,상고이유를 판단한다. 1. 불가벌적 사후행위에 대한 판단「유사수신행위의 규제에...,1066,"['상고', '이유', '를', '판단', '하', 'ㄴ다', '불가벌적 사후행위'...","['상고', '이유', '를', '판단', '하', 'ㄴ다', '불가', '벌', ..."


# 1. 판례내용 전처리

## 1.5 품사 태깅

In [4]:
import konlpy
konlpy.jvm.init_jvm(jvmpath=None, max_heap_size=2048)

[0.003s][warning][os,thread] Attempt to protect stack guard pages failed (0x000000016b604000-0x000000016b610000).
[0.003s][warning][os,thread] Attempt to deallocate stack guard pages failed.


In [5]:
from konlpy.tag import *

kkma = Kkma()
komoran = Komoran()

### 1.5.1 품사 뜯기

In [6]:
tagsets = pd.DataFrame()
N = 67

tagsets["Kkma-기호"] = list(kkma.tagset.keys()) + list("*" * (N - len(kkma.tagset)))
tagsets["Kkma-품사"] = list(kkma.tagset.values()) + list("*" * (N - len(kkma.tagset)))
tagsets["Komoran-기호"] = list(komoran.tagset.keys()) + list("*" * (N - len(komoran.tagset)))
tagsets["Komoran-품사"] = list(komoran.tagset.values()) + list("*" * (N - len(komoran.tagset)))

tagsets

,Kkma-기호,Kkma-품사,Komoran-기호,Komoran-품사
0,EC,연결 어미,EC,연결 어미
1,ECD,의존적 연결 어미,EF,종결 어미
2,ECE,대등 연결 어미,EP,선어말어미
3,ECS,보조적 연결 어미,ETM,관형형 전성 어미
4,EF,종결 어미,ETN,명사형 전성 어미
5,EFA,청유형 종결 어미,IC,감탄사
6,EFI,감탄형 종결 어미,JC,접속 조사
7,EFN,평서형 종결 어미,JKB,부사격 조사
8,EFO,명령형 종결 어미,JKC,보격 조사
9,EFQ,의문형 종결 어미,JKG,관형격 조사


In [7]:
# sorted(kkma.pos(case_fraud.판례내용_전처리[1]), key=lambda x:x[1])

In [8]:
# sorted(komoran.pos(case_fraud.판례내용_전처리[1]), key=lambda x:x[1])

In [9]:
kkma_result = kkma.pos(case_fraud.판례내용_전처리[1])
for i in range(len(kkma_result)):
    if kkma_result[i][1] == 'NNG':
        print(kkma_result[i][0], end = ', ')

상고, 이유, 판단, 피고인, 피고인, 피고인, 피고인, 피고인, 의료법, 위반, 방조, 부분, 피고인, 의료, 법인, 호암, 의료, 재단, 부분, 가, 관련, 법리, 의료, 법인, 명의, 개설, 의료, 기관, 실질적, 비, 의료인, 개설, 운영, 판단, 비, 의료인, 의료, 법인, 명의, 의료, 기관, 개설, 운영, 주도적, 관여, 점, 기본, 비, 의료인, 외형, 형태, 의료, 법인, 탈법, 수단, 악용, 적법, 의료, 기관, 개설, 운영, 가장, 사정, 인정, 사정, 비, 의료인, 실질적, 재산, 출연, 실체, 인정, 의료, 법인, 의료, 기관, 개설, 운영, 수단, 악용, 경우, 의료, 법인, 재산, 부당, 유출, 의료, 법인, 공공성, 비영리, 일탈, 경우, 해당, 인정, 대법원, 선고, 도, 전원, 합의체, 판결, 참조, 원심, 다음, 이유, 피고인, 의료, 법인, 호암, 의료, 재단, 이사장, 원심, 공동, 피고인, 이, 사건, 의료, 기관, 실질적, 개설, 운영, 피고인, 피고인, 피고인, 피고인, 피고인, 부분, 공소, 사실, 유죄, 판단, 심, 판결, 유지, 원심, 공동, 피고인, 은, 사건, 의료, 기관, 개설, 운영, 필요, 자금, 대부분, 조달, 피고인, 의료, 법인, 호암, 의료, 재단, 이하, 사건, 의료, 법인, 다의, 이사장, 취임, 재정, 인사, 업무, 집행, 사건, 의료, 기관, 개설, 운영, 주요, 사항, 주도적, 입장, 처리, 원심, 공동, 피고인, 은, 사건, 의료, 법인, 설립, 당시, 기본, 재산, 액, 합계, 부동산, 보통, 재산, 현금, 출연, 경상북도, 의료, 법인, 설립, 운영, 지침, 의료, 법인, 설립, 허가, 출연금, 기준, 충족, 설립, 허가, 신청, 실제, 허위, 공사, 계약서, 제출, 방법, 부동산, 가액, 현금, 출연, 출연금, 기준, 충족, 재산, 출연, 의료, 법인, 설립, 허가, 영향, 정도, 의료, 법인, 설립, 과정, 하자, 원심, 공동, 피고인, 은, 사건, 의료, 법인, 설립, 과정, 

In [10]:
komoran_result = komoran.pos(case_fraud.판례내용_전처리[1])
for i in range(len(komoran_result)):
    if komoran_result[i][1] == 'NNP':
        print(komoran_result[i][0], end = ', ')

상고, 피고인, 피고인, 피고인, 피고인, 피고인, 피고인, 의료, 법인, 호암, 의료, 의료, 법인, 의료, 법인, 의료, 기관, 형상, 의료, 법인, 의료, 기관, 의료, 법인, 의료, 기관, 의료, 법인, 의료, 법인, 있다, 대법원, 원심, 피고인, 의료, 법인, 호암, 의료, 원심, 피고인, 피고인, 피고인, 피고인, 피고인, 피고인, 공소사실, 심, 원심, 피고인, 은, 의료, 기관, 피고인, 의료, 법인, 호암, 의료, 이하, 의료, 법인, 의료, 기관, 원심, 피고인, 은, 의료, 법인, 보통, 재산, 현금, 경상북도, 의료, 법인, 의료, 법인, 허가, 허가, 허위, 공사, 계약서, 부동산, 현금, 의료, 법인, 허가, 의료, 법인, 원심, 피고인, 은, 의료, 법인, 피고인, 피고인, 피고인, 피고인, 의료, 법인, 이사회, 피고인, 이사회, 의결, 피고인, 원심, 피고인, 은, 의료, 법인, 의료, 법인, 조카, 원심, 피고인, 은, 의료, 법인, 피고인, 피고인, 피고인, 피고인, 피고인, 원심, 피고인, 원심, 피고인, 원심, 원심, 자유심증주의, 의료, 기관, 조범, 피고인, 피고인, 피고인, 피고인, 피고인, 특정경제범죄 가중처벌 등에 관한 법률, 사기, 의료, 기관, 비용, 국민건강보험공단, 비용, 국민건강보험공단, 규칙, 거주지, 특별시, 장, 광역시, 장, 도지사, 군수, 비용, 하면, 서, 도법, 비용, 군수, 국민건강보험공단, 보건, 복지부, 장관, 국민건강보험공단, 규칙, 국민건강보험공단, 명의, 비용, 사유, 한다 시, 도지사, 군수, 국민건강보험공단, 비용, 국민건강보험공단, 국민건강보험공단, 비용, 비용, 군수, 비용, 국민건강보험공단, 국민건강보험공단, 비용, 재원, 비용, 국민건강보험공단, 원심, 비용, 자치, 단체, 국민건강보험공단, 원심, 피고인, 요양, 비용, 비용, 국민건강보험공단, 특정경제범죄 가중처벌 등에 관한 법률, 사기죄, 공소사실, 심, 원심, 원심, 자유심증주의, 사기죄, 이득, 조범, 상고, 오